In [ ]:
from selenium import webdriver  
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.common.by import By
import time 
import numpy as np
import pandas as pd

In [ ]:
# Il link per le ricerche segue sempre lo stesso schema, per tanto abbiamo deciso di costruirlo manualmente attraverso la somma di stringhe 
# Questo ci permetterà di fare un ciclo per lo scraping

link1 = 'https://www.nytimes.com/search?dropmab=true&endDate='
anno = 2020
mese = 7
giorno = 1
ausilio = 0
link2 = '&query=&sort=best&startDate='
link3 = '&types=article'



In [ ]:
# Impostazioni scraping

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Imposta la grandezza della pagina
options.add_argument('window-size=1200x600')

# Lista dove salvare tutti i titoli
titles = []
# Lista dove salvare tutte le keywords
keywords = []

In [ ]:
# Scraping di una pagina
def scraping (url):
    #Inizializzo il driver
    
    # PER WINDOWS: Cambiare webdriver.Chrome in webdriver.Chrome(chrome_options=options)
    # PER MAC: Cambiare webdriver.Chrome in webdriver.Chrome(executable_path='/Users/utente/chromedriver')
    driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)
    
    # Premo il pulsante "accetta" per i cookies, o non riusciremmo ad andare avanti
    # Sospende la navigazione
    time.sleep(2)
    # Preme accetta dei cookies
    cookies = driver.find_element_by_xpath('//button[@class="css-1p91x5t"]')
    cookies.click()
    
    # Ciclo che clicca il pulsante SHOW MORE 
    while True:
    
        try:
            element = driver.find_element_by_xpath('//button[@data-testid="search-show-more-button"]')
            element.click()
            time.sleep(3)
        except TimeoutException:
            break
        except StaleElementReferenceException:
            break
        except NoSuchElementException:
            break
        except ElementClickInterceptedException:
            break
        
    # Creo una lista con tutti gli articoli di questa pagina    
    listArticoli = driver.find_elements_by_xpath('//li[@class="css-1l4w6pd"]')
    
    for articolo in listArticoli :
        key = []
        link = articolo.find_element_by_tag_name('a')
        main_window = driver.current_window_handle
        # Funzione che simula i comandi da tastiera in modo da aprire gli articoli in un'altra pagina
        # PER WINDOWS: Cambiare Keys.COMMAND in Keys.CONTROL 
        # PER MAC: Cambiare Keys.CONTROL in Keys.COMMAND
        link.send_keys(Keys.CONTROL + Keys.RETURN)
        # Cambia la finestra del driver
        driver.switch_to.window(driver.window_handles[1])
    
        
    
        
        tags = driver.find_elements_by_xpath('//meta[@property="article:tag"]')
        # Mi assicuro che siano stati trovati dei tag
        if tags :
            
            # Salva il titolo dell'articolo solo nel caso in cui l'articolo contenga tag
            titoli = driver.find_element_by_xpath('//meta[@property="twitter:title"]')
            title = titoli.get_attribute('content')
            titles.append(title)
            
            for tag in tags:
                keyword = tag.get_attribute('content')
                key.append(keyword)
                tupla = tuple(key)
    
            keywords.append(tupla)
        
    

        # Chiude l'articolo   
        driver.close()
        # Chiude la pagina principale
        driver.switch_to_window(main_window)
        
    driver.close()

In [ ]:
# Per questioni di tempo, abbiamo fatto lo scraping solo dei mesi: luglio, agosto, settembre, ottobre
# Il codice però è costruito per effettuarlo da gennaio fino a novembre

# Utilizzo i mesi per il ciclo. 
while mese < 11:
    # Serie di if per creare man mano il link
    if (mese//10 == 0 and giorno < 10): 
        link= link1 + str(anno) + str(ausilio) + str(mese) + str(ausilio) + str(giorno) + link2 + str(anno) + str(ausilio) + str(mese) + str(ausilio) + str(giorno) + link3
    if (mese//10 == 0 and giorno >= 10):
        link= link1 + str(anno) + str(ausilio) + str(mese)  + str(giorno) + link2 + str(anno) + str(ausilio) + str(mese) + str(giorno) + link3
    if  (mese//10 == 1 and giorno < 10):
        link= link1 + str(anno) + str(mese) + str(ausilio) + str(giorno) + link2 + str(anno) + str(mese) + str(ausilio) + str(giorno) + link3
    if  (mese//10 == 1 and giorno >= 10):
        link= link1 + str(anno) + str(mese) + str(giorno) + link2 + str(anno) + str(mese) + str(giorno) + link3
    
    scraping(link)
    
    if (mese == 4 or mese == 6 or mese == 9):
        if (giorno <= 29):
            giorno += 1
        else:
            mese += 1
            giorno = 1
    elif (mese == 2):
        # Febbraio è stato bisestile quest'anno
        if (giorno <= 28):
            giorno += 1
        else:
            mese +=1
            giorno = 1
    else:
        if (giorno <= 30):
            giorno += 1
        else:
            mese +=1
            giorno = 1


c:\users\gdm17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys
c:\users\gdm17\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: use driver.switch_to.window instead


In [ ]:
# Zip iterator contiene coppie di elementi delle due liste 
# Dict() converte zip iterator in un dizionario con coppie chiave-valore

zip_iterator = zip(titles, keywords)
my_dictionary = dict(zip_iterator)

# Creo il mio dataset
df = pd.DataFrame(list(my_dictionary.items()),columns = ['Titles','Keywords']) 

# Creo il CSV
df.to_csv(r'C:\Users\Utente\Desktop\datasetArticoliKeywords.csv', encoding='utf-8-sig', index = False)

In [ ]:
columns = ["A", "B"]
dfN2N = pd.DataFrame (columns= columns)

In [ ]:
counter = 0
for n in df['Keywords']:
    for c in n:
        for z in range (n.index(c)+1,len(n)):
            dfN2N.at[counter,'A'] = c
            dfN2N.at[counter,'B'] = n[z]
            counter += 1

In [ ]:
# Usiamo una semplice lista per creare il vocabolario
# VOCABOLARIO: contiene tutte le parole diverse tra loro

vocabolario = []
for i in keywords:
    for n in i:
        vocabolario.append(n)
        
vocabolario = list(set(vocabolario))

In [ ]:
numericalKeywords = []
count = 0
for i in vocabolario:
    numericalKeywords.append(count)
    count += 1

    

In [ ]:
zip_iterator = zip(vocabolario, numericalKeywords)
my_dictionary = dict(zip_iterator)

In [ ]:
# Stampa le chiavi corrispondenti alle parole in base al mio vocabolario
Keys_id = []

for lista in df['Keywords']:
    new = []
    for elm in lista:
        for key,value in my_dictionary.items():
            if (elm == key):
                new.append(value)
                tupla = tuple(new)
                
    Keys_id.append(tupla)

In [ ]:
df['Keys_Id'] = Keys_id
df.to_csv(r'C:\Users\Utente\Desktop\DatasetKeywords-ID.csv', index = False)

In [ ]:
columns = ["A", "B"]
dfN2NN = pd.DataFrame (columns= columns)

In [ ]:
counter = 0
for n in df['Keys_Id']:
    for c in n:
        for z in range (n.index(c)+1,len(n)):
            dfN2NN.at[counter,'A'] = c
            dfN2NN.at[counter,'B'] = n[z]
            counter += 1

In [ ]:
# Trasformiamo le coppie in tuple
nuovaLista = []

l=len(dfN2NN['A'])

# Per i (indice) fino a lunghezza massima della lista
for i in range(l):
        coppie = []
        # L'indice si aggiorna sia per A sia per B
        coppie.append(dfN2NN['A'][i])
        coppie.append(dfN2NN['B'][i])
        tupla = tuple(coppie)
        nuovaLista.append(tupla)

In [ ]:
# Eseguo il calcolo delle frequenze
frequenze = {}
for item in nuovaLista:
    if (item in frequenze):
        frequenze[item] += 1
    else:
        frequenze[item] = 1
                     

In [ ]:
# Trasformo in una lista tutte le chiavi del vocabolario
# Ottengo quindi una lista con tutti gli elementi DIVERSI tra loro

y = list(frequenze.keys()) # Qui le ripetizioni vengono eliminate

In [ ]:
rimuoviElementi = []   # Lista che conterrà gli elementi da rimuovere

for i in y:
    for c in range(y.index(i)+1,len(y)):
            if (i[0] == y[c][1] and i[1] == y[c][0]):  
                rimuoviElementi.append(i)          # Inserisce uno dei due elementi nella lista 'da rimuovere'
                for key,value in frequenze.items():
                    v = 0
                    if(i == key):
                        v = value
                        for key2,value2 in frequenze.items():
                            if(y[c] == key2): 
                                frequenze[key2] += v  # Somma il valore delle chiavi dei due elementi

# In questo modo un elemento avrà la somma delle due frequenze mentre l'altro sarà inserito nella lista degli elementi da rimuovere

In [ ]:
# Prendo i 'valori' del dizionario

listaFrequenze = []

for elm in nuovaLista:                # Uso nuovaLista in cui ci sono tutti gli elementi
    for key,value in frequenze.items():   # Uso il vocabolario frequenze in cui ho tutti i valori aggiornati
        if (elm == key):
            listaFrequenze.append(value)

In [ ]:
dfN2NN['Frequenze'] = listaFrequenze

In [ ]:
dfQ = dfN2NN.copy()

In [ ]:
for n in rimuoviElementi:
    for c in range (0, len(dfN2NN)):
        if (n[0] == dfN2NN.at[c, 'A'] and n[1] == dfN2NN.at[c, 'B']):
            dfQ.drop(c, inplace = True)

In [ ]:
datasetP = dfQ.drop_duplicates()

In [ ]:
datasetF = datasetP.reset_index(drop = True)

In [ ]:
datasetF.head()

,A,B,Frequenze
0,7364,3512,2
1,18570,3512,1
2,13728,7040,6
3,7040,1480,1
4,7040,4429,1


In [ ]:
datasetF.to_csv(r'C:\Users\Utente\Desktop\DatasetFinale.csv', index = False)